# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [41]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import re

In [42]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [43]:
! mongoimport --type json -d uk_food -c establishments --drop --jsonArray Resources/establishments.json

2024-12-25T22:49:14.232-0600	connected to: mongodb://localhost/
2024-12-25T22:49:14.233-0600	dropping: uk_food.establishments
2024-12-25T22:49:15.588-0600	39779 document(s) imported successfully. 0 document(s) failed to import.


In [44]:
# confirm that our new database was created
# one line pprint
pprint(mongo.list_database_names())

['admin',
 'autosaurus',
 'classDB',
 'config',
 'epa',
 'epa_db',
 'garden_db',
 'local',
 'met',
 'petsitly_marketing',
 'travel_db',
 'uk_food']


In [45]:
# confirm that our new database was created
databases = mongo.list_database_names()
for database in databases:
    print(database)

admin
autosaurus
classDB
config
epa
epa_db
garden_db
local
met
petsitly_marketing
travel_db
uk_food


In [46]:
# assign the uk_food database to a variable name
db = mongo['uk_food'] # or
# db = mongo.uk_food

In [47]:
# review the collections in our new database
pprint(db.list_collection_names())

['establishments']


In [49]:
# review a document in the establishments collection
db.establishments.find_one()

{'_id': ObjectId('676ce04a3173a8977107c64d'),
 'FHRSID': 254719,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT15 6DY',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2022-03-24T00:00:00',
 'LocalAuthorityCode': '182',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'scores': {'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.387974', 'latitude': '51.152225'},
 'RightToReply': '',
 'Distance': 4587.347174863443,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount': 0,
  'returncode

In [50]:
# assign the collection to a variable
establishments = db['establishments'] # or
# establishments = db.establishments

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [53]:
print(establishments.count_documents({}))

39779


In [54]:
# Create a dictionary for the new restaurant data
new_stores = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [55]:
# Insert the new restaurant into the collection
db.establishments.insert_one(new_stores)

InsertOneResult(ObjectId('676ce04b2dcd910c4d489ed6'), acknowledged=True)

In [56]:
# Check that the new restaurant was inserted
print(establishments.count_documents({}))

39780


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [58]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
fields = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
results = db.establishments.find_one(query, fields)
print(results)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [60]:
# Update the new restaurant with the correct BusinessTypeID
db.establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set": {"BusinessTypeID": 1}}
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [61]:
# Confirm that the new restaurant was updated
result = db.establishments.find_one({"BusinessName": "Penang Flavours"})
pprint(result)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('676ce04b2dcd910c4d489ed6'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [63]:
# using regex
query = {"LocalAuthorityName": {"$regex": "Dover", "$options": "i"}}  # "$options": 'i' for case-insensitive
results = db.establishments.delete_many(query)
results

DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [64]:
# Delete all documents where LocalAuthorityName is "Dover"
# query = {"LocalAuthorityName": "Dover"}
# results = db.establishments.delete_many(query)
# results

In [65]:
# Check if any remaining documents include Dover
results = db.establishments.find({"LocalAuthorityName": "Dover"})
dover_est = []
for result in results:
    dover_est.append(result)
print(dover_est)

[]


In [66]:
# count the number of documents that remain in the collection using count_documents
count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"The number of restruants that are in Dover: ",count)

The number of restruants that are in Dover:  0


In [67]:
# Check that other documents remain with 'find_one'
db.establishments.find_one()

{'_id': ObjectId('676ce04a3173a8977107c934'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [85]:
# Change the data type from String to Decimal for longitude and latitude
db.establishments.update_many(
    {},
    [
        {"$set": {
            "geocode.longitude": {"$toDecimal": "$geocode.longitude"},
            "geocode.latitude": {"$toDecimal": "$geocode.latitude"}
                }
        }
    ]
)

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [88]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [90]:
# Change the data type from String to Integer for RatingValue
db.establishments.update_many(
    {},
    [
        {"$set": {
            "RatingValue": {"$toInt": "$RatingValue"}
                }
        }
    ]
)

UpdateResult({'n': 38786, 'nModified': 34695, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [92]:
# Check that the coordinates and rating value are now numbers
db.establishments.find_one()

{'_id': ObjectId('676ce04a3173a8977107c934'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': 5,
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': Decimal128('1.195625'),
  'latitude': Decimal128('51.083812')},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'e